# Postprocessors #

Various postprocessors are included, and verified below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Component mass ##

The ComponentMassPostProcessor calculates the total mass of a speciifed component over a single or multiple phases.

### Immiscible fluids ###

The first test is a 1 x 1 x 1 mesh with unit volume, porosity 0.25, liquid saturation 0.7 and gas saturation 0.3. Liquid is water at 1 MPa and 100 C, and gas is CO$_2$ at the same pressure and temperature. No mutual solubility included.

At these conditions, the density of water is 958.77 kg/m^3, and the density of CO$_2$ is 14.525 kg/m^3 (http://webbook.nist.gov).

In [22]:
water_density = 958.77
co2_density = 14.525
volume = 1
porosity = 0.25
water_saturation = 0.7
gas_saturation = 0.3

water_mass =  water_density * volume * water_saturation * porosity
co2_mass =  co2_density * volume * gas_saturation * porosity

print "Total water mass is", water_mass
print "Total co2 mass is", co2_mass

Total water mass is 167.78475
Total co2 mass is 1.089375


The MOOSE mass are

In [36]:
moosedata1 = pd.read_csv('../../../tests/postprocessors/gold/MassPostProcessor1.csv', header = 0)

print "Total water mass is", moosedata1.totalh2omass[0]
print "Total co2 mass is", moosedata1.totalco2mass[0]

Total water mass is 167.777419133
Total co2 mass is 1.08255204907


The error in these results is

In [37]:
print "Relative error in water mass " ,(water_mass - moosedata1.totalh2omass[0])/water_mass
print "Relative error in co2 mass " ,(co2_mass - moosedata1.totalco2mass[0])/co2_mass

Relative error in water mass  4.36920928152e-05
Relative error in co2 mass  0.00626317928473


This error is attributed to the error in density for water and co2 used in the fluid property correlations. If we use the MOOSE values for densities, the water and co2 mass are identical to those reported my the postprocessor.

In [38]:
print "Total water mass is", moosedata1.liquid_density[0] * volume * water_saturation * porosity
print "Total co2 mass is", moosedata1.gas_density[0] * volume * gas_saturation * porosity

Total water mass is 167.777419133
Total co2 mass is 1.08255204907


###  Including solubility ###

The second test is also 1 x 1 x 1 mesh with unit volume, porosity 0.25, liquid saturation 0.7 and gas saturation 0.3. Liquid is water at 1 MPa and 100 C, and gas is CO$_2$ at the same pressure and temperature. 

In this case, however, the WaterCO2 fluid state is used, which calculates the mass fraction of CO2 in liquid and H2O in gas. The sum of the mass of each component in each phase should give the total masses calculated by the postprocessor.


In [42]:
moosedata2 = pd.read_csv('../../../tests/postprocessors/gold/MassPostProcessor2.csv', header = 0)

print "Total water mass is", moosedata2.totalh2omass[0]
print "Total co2 mass is", moosedata2.totalco2mass[0]

Total water mass is 167.26803928
Total co2 mass is 1.69507212873


Calculating the mass in each phase and comparing it to the value provided by the postprocessor.

In [57]:
h2oliquid = moosedata2.xh2ol[0] * moosedata2.liquid_density[0] * water_saturation * porosity
h2ogas = moosedata2.xh2og[0] * moosedata2.gas_density[0] * gas_saturation * porosity
co2liquid = moosedata2.xco2l[0] * moosedata2.liquid_density[0] * water_saturation * porosity
co2gas = moosedata2.xco2g[0] * moosedata2.gas_density[0] * gas_saturation * porosity

print "Caculated total H2O mass =", h2oliquid + h2ogas, ", MOOSE result =", moosedata2.totalh2omass[0]
print "Caculated total CO2 mass =", co2liquid + co2gas, ", MOOSE result =", moosedata2.totalco2mass[0]

Caculated total H2O mass = 167.26803928 , MOOSE result = 167.26803928
Caculated total CO2 mass = 1.69507212873 , MOOSE result = 1.69507212873
